In [ ]:
import numpy as np
from scipy import linalg
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode, iplot
from ipywidgets import interactive

init_notebook_mode()

def fermi_gas_1D(n=1):
    n_down = (n + 1) // 2
    n_up = n // 2
    momenta = np.linspace(-10, 10, 101)
    energies = momenta**2
    data = [go.Scatter(x=momenta, y=energies, hoverinfo='none', line={'color':'black'})]
    discrete_momenta = np.arange(-19, 20)
    discrete_energies = discrete_momenta**2
    sort_order = np.argsort(discrete_energies)
    discrete_momenta = discrete_momenta[sort_order]
    discrete_energies = discrete_energies[sort_order]
    data.append(go.Scatter(x=discrete_momenta[:n_down], y=discrete_energies[:n_down] - 2,
                           marker=go.Marker(color='red', symbol='circle'), line={'width':0}))
    data.append(go.Scatter(x=discrete_momenta[:n_up], y=discrete_energies[:n_up] + 3,
                           marker=go.Marker(color='blue', symbol='circle'), line={'width':0}))
    layout = go.Layout(
            showlegend=False,
            autosize=False,
            width=600,
            height=300,
            xaxis={'title':'k'},
            yaxis={'title':'E'})
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

fermi_gas_1D = interactive(fermi_gas_1D, n=(1, 39))

def plot_dos(dim=1, mu=1., T=0.001):
    E = np.linspace(0, 2, 300)
    n = E**((dim - 2) / 2) / (np.exp((E - mu) / T) + 1)
    data = [go.Scatter(x=E, y=n, hoverinfo='none', line={'color':'black'})]
    layout = go.Layout(
            showlegend=False,
            autosize=False,
            width=600,
            height=300,
            xaxis={'title':'E'},
            yaxis={'title':'n'})
    iplot(go.Figure(data=data, layout=layout), show_link=False)

dos = interactive((lambda dim=1., mu=1.: plot_dos(dim, mu)), dim=(1, 3), mu=(0., 2.))

dos_finite = interactive(plot_dos, dim=(1, 3), mu=(0., 2.), T=(0.001, .1, 0.01))

# Free electron model
By Anton Akhmerov *(also it's my very first lecture ever today!)*

This lecture:
* Fermi surface
* Fermi energy
* Fermi velocity
* Electron heat capacitance

Next lecture:  
Scattering and magnetic field

# Electrons

Q: *In which ways are electrons different from phonons?*

* They have electric charge.
* You cannot create new electrons, while you can create phonons  
  (So they obey different statistics, fermions, not bosons)
* They have a mass:  
  $E = p^2/2m$ for electrons instead of $E = v|p|$ for (acoustic) phonons
  
*NB: Electrons are my favorite particle*

Q: *What is similar between electrons and phonons?*

* Both interact with lattice
* Both can store extra energy when the system is heated

Focus on differences first, forget the lattice exists (imagine the nuclei form an average potential for electrons).

# Density of states

Plan: count states in a big box

Electrons obey Schrödinger equation:

$$ H = -\frac{\hbar^2}{2m}\left(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}\right) $$

Take $L\times L \times L$ box with periodic boundary conditions: $\psi(L, y, z) = \psi(0, y, z)$, same for $y$ and $z$.

Wave vectors are now quantized: $k_{x,y,z} = 2\pi n_{x, y, z}/ L$

...and energies as well: $E(n_x, n_y, n_z) = \frac{(2\pi\hbar)^2}{mL^2}(n_x^2 + n_y^2 + n_z^2)$

# Lowest energy state of many electrons

Let's start with a 1D illustration:

In [ ]:
fermi_gas_1D

## Fermi-everything

* The lowest energy state is called **Fermi sea**, and the top of Fermi sea is **Fermi surface**
* All the states below **Fermi energy** ($E_\text{F}$) are filled, those above are empty
* This means $|k| < k_\text{F}$, the **Fermi momentum**

## Counting the states

Number of states with $|k| < k$:
* 1D: $N = \frac{2 \times 2 k}{2\pi/L} = \frac{4 k V}{\pi}$
* 2D: $N = \frac{2 \pi k^2}{(2\pi/L)^2} = \frac{k^2 V}{4\pi}$
* 3D: $N = \frac{2 (4/3) \pi k^3 }{ (2\pi/L)^3} = \frac{k^3 V}{3\pi^2}$

## Density of states

We need to get:

$$g(E) \equiv \frac{dN}{dE} = \frac{dN}{dk}\frac{dk}{dE}.$$

From Schrödinger equation:

$$k = \hbar^{-1}\sqrt{2mE};\quad \frac{dk}{dE}=\hbar^{-1}\sqrt{m/2E}$$

So:
* 1D: $g(E) = \frac{4 V}{\pi} \frac{dk}{dE} \sim E^{-1/2}$
* 2D: $g(E) = \frac{k V}{2\pi} \frac{dk}{dE} \sim \text{const}$
* 3D: $g(E) = \frac{k^2 V}{\pi^2} \frac{dk}{dE} \sim E^{1/2}$

## Number of states with each energy

In [ ]:
dos

# Effects of finite temperature

Fermi-Dirac distribution (check wiki: [WP:Fermi Distribution](https://en.wikipedia.org/wiki/Fermi%E2%80%93Dirac_statistics#Derivations_of_the_Fermi.E2.80.93Dirac_distribution)):

$ f(E, T) = \frac{1}{e^{(E - \mu)/k_\text{B}T} + 1} $

At finite temperature $n(E) = g(E)\times f(E, T)$

In [ ]:
dos_finite

# Heat capacitance:

$n \approx (1/2) g(E_F) \times k_B T$ electrons have their energy increased by $\delta E \sim k_B T$

$$E(T) - E(0) \approx g(E_F) (k_b T)^2$$

So:
$$ C_{v, e} \approx g(E_F) k_b^2 T = \ldots = \frac{3}{2} N k_B \frac{T}{T_F}$$

($T_F = E_F/ k_B$ is **Fermi temperature**)

At room temperature $C_{v, f} = 3Nk_B \gg C_{v, e}$.

At $T \to 0$, $C_{v, f} \sim T^3$, while $C_{v, f} \sim T$.